In [1]:
import os
import openai
 
openai.api_key = ""
def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [2]:
import yaml
import alfworld
import alfworld.agents.environment
with open('tests/assets/base_config.yaml') as reader:
    config = yaml.safe_load(reader)
    
print('hello wolrd')
split = "eval_out_of_distribution"

env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
env = env.init_env(batch_size=1)

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]    
    return ob

hello wolrd
Initializing AlfredTWEnv...
Checking for solvable games...


100%|██████████| 341/341 [00:00<00:00, 1206.58it/s]

Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games


In [3]:
from discussion_agents.cog.prompts.react import REACT_ALFWORLD_PROMPTS_EXAMPLE
d = REACT_ALFWORLD_PROMPTS_EXAMPLE

In [4]:

def alfworld_run(prompt, to_print=True, ob=''):
    init_prompt = prompt + ob + '\n>'
    prompt = ''
    if to_print:
        print(ob)
    for i in range(1, 50):
        action = llm(init_prompt + prompt, stop=['\n']).strip()
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        print(observation, reward, done)
        if action.startswith('think:'):
            observation = 'OK.'
        if to_print:
            print(f'Act {i}: {action}\nObs {i}: {observation}')
        prompt += f' {action}\n{observation}\n>'
        if done:
            return reward
    return 0

In [31]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}
cnts = [0] * 6
rs = [0] * 6


ob, info = env.reset()
ob = '\n'.join(ob[0].split('\n\n')[1:])
name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])
print(name)
for i, (k, v) in enumerate(prefixes.items()):
    if name.startswith(k):
        prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0'] + '\nHere is the task.\n'
 



pick_and_place_simple-Watch-None-Safe-219/trial_T20190907_074556_124850


In [6]:
from langchain_community.chat_models import ChatOpenAI


llm = ChatOpenAI(openai_api_key="")

In [7]:
from discussion_agents.cog.agent.react import ReActAgent


test_run = ReActAgent(llm = llm)

In [15]:
v

'puttwo'

In [17]:
examples = prompt

In [18]:
examples

'Interact with a household to solve a task. Here are two examples.\nYou are in the middle of a room. Looking quickly around you, you see a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 1, a diningtable 3, a diningtable 2, a diningtable 1, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sidetable 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: clean some apple and put it in sidetable.\n> think: To solve the task, I need to find and take an apple, then clean it with sinkbasin, then put it in sidetable.\nOK.\n> think: First I need to find an apple. An apple is more likely to appear in fridges (1), diningtable (1-3), sidetable (1), countertop (1), sinkbasin (1), garbagecan (1). I can check one by one, starting with fridge 1.\nOK.\n> go to fridge 1\nThe fridge 1 is closed.\n> open fridge 1\nYou open the fridge 1. The fridge 1 is open. In it, you see a lettuce 2, a mug 2, a p

In [19]:
from discussion_agents.cog.prompts.react import REACT_ALFWORLD_INSTRUCTION

In [32]:

test_run.set_Alfworld()
observation = None
done = False
for i in range(0,50):
    ans = test_run.step(question=ob, examples=examples , prompt_template=REACT_ALFWORLD_INSTRUCTION , env_output=observation)
    ans = ans[0].strip()
    observation, reward, done, info = env.step([ans])
    observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
    print(observation, reward, done)
    if ans.startswith('think:'):
        observation = 'OK.'
    if True:
        print(f'Act {i}: {ans}\nObs {i}: {observation}')
    prompt += f' {ans}\n{observation}\n>'
    if done:
        break


Nothing happens. False False
Act 0: think: To solve the task, I need to find some watches and put them on the safe.
Obs 0: OK.
Nothing happens. False False
Act 1: think: First, I need to find some watches. Watches are more likely to be found in drawers (1-5), shelves (1-12), cabinets (1-4), and maybe on the dresser (1). I can start by checking the drawers.
Obs 1: OK.
The drawer 1 is closed. False False
Act 2: go to drawer 1
Obs 2: The drawer 1 is closed.
You open the drawer 1. The drawer 1 is open. In it, you see a watch 1. False False
Act 3: open drawer 1
Obs 3: You open the drawer 1. The drawer 1 is open. In it, you see a watch 1.
You pick up the watch 1 from the drawer 1. False False
Act 4: take watch 1 from drawer 1
Obs 4: You pick up the watch 1 from the drawer 1.
The safe 1 is closed. False False
Act 5: go to safe 1
Obs 5: The safe 1 is closed.
You open the safe 1. The safe 1 is open. In it, you see a vase 3, and a vase 2. False False
Act 6: open safe 1
Obs 6: You open the safe 1

In [30]:
test_run.reset()

In [16]:
string = "think: I did not find a cool tomato in the fridge 1. Next, I will check the countertop 1-3."

In [19]:
not(string.startswith('think'))

False